In [753]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [649]:
from src.models import hmm, decoders

In [650]:
import numpy as np

In [651]:
observation_states = ['committed','ambivalent']

In [652]:
hidden_states = hidden_states = ['R01','R21']

In [653]:
use_case_one_data = np.load('./data/UserCase-Lecture.npz')

In [654]:
use_case_one_hmm = hmm.HiddenMarkovModel(observation_states,
                                         hidden_states,
                      use_case_one_data['prior_probabilities'], 
                      use_case_one_data['transition_probabilities'],
                      use_case_one_data['emission_probabilities']) 

In [655]:
use_case_one_viterbi = decoders.ViterbiAlgorithm(use_case_one_hmm)

In [656]:
observation_states = use_case_one_hmm.observation_states
observation_states_dict = use_case_one_hmm.observation_states_dict
hidden_states = use_case_one_hmm.hidden_states
hidden_states_dict = use_case_one_hmm.hidden_states_dict
print(observation_states)
print(observation_states_dict)
print(hidden_states)
print(hidden_states_dict)

['committed', 'ambivalent']
{'committed': 0, 'ambivalent': 1}
['R01', 'R21']
{0: 'R01', 1: 'R21'}


In [743]:
prior_prob = use_case_one_hmm.prior_probabilities
transition_prob = use_case_one_hmm.transition_probabilities
emission_prob = use_case_one_hmm.emission_probabilities
print(prior_prob)
print(prior_prob.shape)
print(transition_prob.shape)
print(emission_prob.shape)

[0.67 0.33]
(2,)
(2, 2)
(2, 2)


In [742]:
decode_observation_states = use_case_one_data['observation_states']
print(decode_observation_states)

['committed' 'committed' 'ambivalent' 'ambivalent' 'ambivalent'
 'committed']


In [745]:
print(len(hidden_states))
print(len(observation_states))
print(len(decode_observation_states))
print(path)
print(best_path)

2
2
6
[[0.536      0.132     ]
 [0.34304    0.04288   ]
 [0.0548864  0.0411648 ]
 [0.00878182 0.01481933]
 [0.00140509 0.00533496]
 [0.00170719 0.00128039]]
[[0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [1 1]]


In [551]:
print(decode_observation_states[0])
start_obs = observation_states_dict[decode_observation_states[0]]
print(start_obs)

committed
0


In [553]:
delta = np.multiply(prior_prob, emission_prob[:, start_obs])
delta

array([0.536, 0.132])

In [554]:
#p_d_t = np.multiply(delta, transition_prob)
#print(p_d_t)
p_d_t_T = np.multiply(delta, transition_prob.T)
print(p_d_t_T)

[[0.4288 0.0528]
 [0.1072 0.0792]]


In [468]:
p_d_t_T_e = np.multiply(p_d_t_T.T, emission_prob[:, start_obs])
print(p_d_t_T_e)

[[0.51353293 0.06419162]
 [0.06323353 0.04742515]]


In [392]:
x

5

In [469]:
prod_delta_trans = np.multiply(delta, transition_prob.T)
print(prod_delta_trans)
print(prod_delta_trans.T)
print(emission_prob[:,0])
prod_delta_trans_emission = np.multiply(prod_delta_trans.T, emission_prob[:,0])
print(prod_delta_trans_emission)
print(prod_delta_trans_emission.T)

max_hidden_prob = np.max(prod_delta_trans_emission, axis=0)
max_hidden_prob_idx = np.argmax(prod_delta_trans_emission, axis=0)
print(np.argmax(prod_delta_trans_emission, axis=0))
print(np.max(prod_delta_trans_emission, axis=0))

print(np.argmax(prod_delta_trans_emission.T, axis=1))
print(np.max(prod_delta_trans_emission.T, axis=1))

[[0.64191617 0.07904192]
 [0.16047904 0.11856287]]
[[0.64191617 0.16047904]
 [0.07904192 0.11856287]]
[0.8 0.4]
[[0.51353293 0.06419162]
 [0.06323353 0.04742515]]
[[0.51353293 0.06323353]
 [0.06419162 0.04742515]]
[0 0]
[0.51353293 0.06419162]
[0 0]
[0.51353293 0.06419162]


In [549]:
path = np.zeros((len(decode_observation_states), 
                len(hidden_states)))
path[0,:] = [hidden_state_index for hidden_state_index in range(len(hidden_states))]
best_path = np.zeros(len(decode_observation_states))
best_path[0] = path[0][np.argmax(delta)]
print(path)
print(best_path)

[[0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[0. 0. 0. 0. 0. 0.]


In [527]:
for trellis_node in range(1, len(decode_observation_states)):
    curr_obs = observation_states_dict[decode_observation_states[trellis_node]]
    
    prod_delta_trans = np.multiply(delta, transition_prob.T)
    prod_delta_trans_emission = np.multiply(prod_delta_trans.T, emission_prob[:, curr_obs])
    
    max_prob = np.max(prod_delta_trans_emission, axis=0)
    max_prob_idx = np.argmax(prod_delta_trans_emission, axis=0)
    #rows are hidden state, columns are observed state
    max_prob_scaled = max_prob / np.sum(max_prob)
    
    path[trellis_node,:] = max_prob_idx
    best_path[trellis_node-1] =np.argmax(max_prob_scaled)
    
    delta = np.multiply(prior_prob, emission_prob[:, curr_obs])
    

In [528]:
best_path

array([0, 0, 1, 1, 0, 0], dtype=int32)

In [529]:
path

array([[0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [741]:
T = len(decode_observation_states)
N = len(hidden_states)

path = np.zeros((T, N))


best_path = np.zeros((T, N), dtype=int)

start_obs = observation_states_dict[decode_observation_states[0]]

# Initialize the first column of the Viterbi matrix
delta = np.multiply(prior_prob, emission_prob[:, start_obs])
path[0, :] = delta

# Loop over the remaining columns of the Viterbi matrix
for trellis_node in range(1, len(decode_observation_states)):
    curr_obs = observation_states_dict[decode_observation_states[trellis_node]]
    prev_delta = path[trellis_node - 1, :]
    for hidden_state in range(len(hidden_states)):
        # probabilities of transitioning to this hidden state given previous delta and current observation state
        prod_delta_trans = np.multiply(prev_delta, transition_prob[:, hidden_state]) 
        print(prod_delta_trans, hidden_state)
        
        max_prob_idx = np.argmax(prod_delta_trans) #index of most likely previous state with max prob
        print(max_prob_idx)
        max_prob_val = prod_delta_trans[max_prob_idx] #get max prob value to multiply with emission probabilities
        print(max_prob_val)

        # Update the path and best path
        prod_delta_trans_emission = np.multiply(max_prob_val, emission_prob[hidden_state, curr_obs])
        path[trellis_node,hidden_state] = np.max(prod_delta_trans_emission) #add new delta to path for given hidden state
        print(path)
        
        best_path[trellis_node,hidden_state] = max_prob_idx #store index of highest prob
        
# get final state by returning index of max value in last row of path matrix
final_state = np.argmax(path[-1, :])

# Follow the backpointers to reconstruct the most likely sequence of states
best_hidden_state_path = [final_state]
for state in range(len(decode_observation_states) - 1, 0, -1):
    prev_state = best_path[state, best_hidden_state_path[-1]]
    best_hidden_state_path.append(prev_state)
best_hidden_state_path.reverse()


[0.4288 0.0528] 0
0
0.42880000000000007
[[0.536   0.132  ]
 [0.34304 0.     ]
 [0.      0.     ]
 [0.      0.     ]
 [0.      0.     ]
 [0.      0.     ]]
[0.1072 0.0792] 1
0
0.10720000000000002
[[0.536   0.132  ]
 [0.34304 0.04288]
 [0.      0.     ]
 [0.      0.     ]
 [0.      0.     ]
 [0.      0.     ]]
[0.274432 0.017152] 0
0
0.27443200000000006
[[0.536     0.132    ]
 [0.34304   0.04288  ]
 [0.0548864 0.       ]
 [0.        0.       ]
 [0.        0.       ]
 [0.        0.       ]]
[0.068608 0.025728] 1
0
0.06860800000000002
[[0.536     0.132    ]
 [0.34304   0.04288  ]
 [0.0548864 0.0411648]
 [0.        0.       ]
 [0.        0.       ]
 [0.        0.       ]]
[0.04390912 0.01646592] 0
0
0.04390912000000002
[[0.536      0.132     ]
 [0.34304    0.04288   ]
 [0.0548864  0.0411648 ]
 [0.00878182 0.        ]
 [0.         0.        ]
 [0.         0.        ]]
[0.01097728 0.02469888] 1
1
0.024698880000000003
[[0.536      0.132     ]
 [0.34304    0.04288   ]
 [0.0548864  0.0411648 ]
 

In [729]:
print(best_hidden_state_path)

[0, 0, 1, 1, 1, 0]


In [730]:
hidden_state_path = [hidden_states_dict[i] for i in best_hidden_state_path]

In [731]:
hidden_state_path

['R01', 'R01', 'R21', 'R21', 'R21', 'R01']

In [754]:
observation_states = ['on-time','late'] 

# index annotation hidden_states=[i,j]
hidden_states = ['no-traffic','traffic']

# PONDERING QUESTION: How would a user define/compute their own HMM instantiation inputs to decode the hidden states for their use case observations?
use_case_one_data = np.load('./data/UserCase-One.npz')

# Instantiate submodule class models.HiddenMarkovModel with
# observation and hidden states and prior, transition, and emission probabilities.
use_case_one_hmm = hmm.HiddenMarkovModel(observation_states,
                                     hidden_states,
                  use_case_one_data['prior_probabilities'], # prior probabilities of hidden states in the order specified in the hidden_states list
                  use_case_one_data['transition_probabilities'], # transition_probabilities[:,hidden_states[i]]
                  use_case_one_data['emission_probabilities']) 

In [755]:
use_case_one_viterbi = decoders.ViterbiAlgorithm(use_case_one_hmm)

In [756]:
decode_observation_states = use_case_one_data['observation_states']
print(decode_observation_states)

['on-time' 'on-time' 'late' 'late' 'late' 'on-time']


In [757]:
test_seq = use_case_one_viterbi.best_hidden_state_sequence(decode_observation_states)

In [759]:
test_seq

array(['no-traffic', 'no-traffic', 'traffic', 'traffic', 'traffic',
       'no-traffic'], dtype='<U10')